In [ ]:
!pip install --quiet pandas numpy matplotlib plotly geopandas folium scikit-learn prophet pyngrok streamlit pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.graph_objects as go

In [ ]:
global_df = pd.read_csv('/content/Top 1000 technology companies.csv')
startup_df = pd.read_csv('/content/startup_funding.csv')
ict_df = pd.read_csv('/content/ICT_Subdimension_Dataset.csv')

In [ ]:
def normalize_columns(df):
    df.columns = [c.strip().replace('\n', '').replace('\r', '').replace('  ', ' ') for c in df.columns]
    df.columns = [c.strip() for c in df.columns]
    return df

global_df = normalize_columns(global_df)
startup_df = normalize_columns(startup_df)
ict_df = normalize_columns(ict_df)

In [ ]:
print("\nNormalized column names loaded successfully:")

print("Global:", global_df.columns.tolist())
print("Startup:", startup_df.columns.tolist())
print("ICT:", ict_df.columns.tolist())


Normalized column names loaded successfully:
Global: ['Ranking', 'Company', 'Market Cap', 'Stock', 'Country', 'Sector', 'Industry']
Startup: ['Sr No', 'Date dd/mm/yyyy', 'Startup Name', 'Industry Vertical', 'SubVertical', 'City Location', 'Investors Name', 'InvestmentnType', 'Amount in USD', 'Remarks']
ICT: ['City', 'Year', 'Household Internet Access (%)', 'Fixed Broadband Subscriptions (%)', 'Wireless Broadband Subscriptions (%)', 'Wireless Broadband Coverage 3G (%)', 'Wireless Broadband Coverage 4G (%)', 'Smart Water Meters (%)', 'Smart Electricity Meters (%)', 'Dynamic Public Transport Information (%)', 'Traffic Monitoring (%)', 'Availability of WIFI in Public Areas (count)', 'Water Supply ICT Monitoring (%)', 'Drainage / Storm Water System ICT Monitoring (%)', 'Electricity Supply ICT Monitoring (%)', 'Demand Response Penetration (%)', 'Intersection Control (%)', 'Open data (%)', 'e-Government (%)', 'Public Sector e-procurement (%)']


**GLOBAL TECH DATA CLEANING**

In [ ]:
def convert_market_cap(v):
    try:
        s = str(v).replace('$', '').replace(',', '').strip()
        if s.endswith('T'):
            return float(s[:-1]) * 1000
        elif s.endswith('B'):
            return float(s[:-1])
        else:
            return float(s)
    except:
        return np.nan

In [ ]:
market_col = [c for c in global_df.columns if 'market' in c.lower()][0]
country_col = [c for c in global_df.columns if 'country' in c.lower()][0]
sector_col = [c for c in global_df.columns if 'sector' in c.lower()][0]
industry_col = [c for c in global_df.columns if 'industry' in c.lower()][0]

global_df['MarketCap_BillionUSD'] = global_df[market_col].apply(convert_market_cap)
global_df['Country'] = global_df[country_col].astype(str).str.strip()
global_df['Sector'] = global_df[sector_col].astype(str).str.strip()
global_df['Industry'] = global_df[industry_col].astype(str).str.strip()
global_df.dropna(subset=['MarketCap_BillionUSD'], inplace=True)

**STARTUP FUNDING DATA CLEANING**

In [ ]:
def parse_amount(v):
    try:
        s = str(v).replace(',', '').replace('$', '').strip()
        if s.lower() in ['undisclosed', 'nan', 'none', 'na', '-', '']:
            return np.nan
        return float(s)
    except:
        return np.nan

amt_col = [c for c in startup_df.columns if 'amount' in c.lower()][0]
date_col = [c for c in startup_df.columns if 'date' in c.lower()][0]
city_col = [c for c in startup_df.columns if 'city' in c.lower()][0]
industry_col = [c for c in startup_df.columns if 'industry' in c.lower()][0]

startup_df['AmountUSD'] = startup_df[amt_col].apply(parse_amount)
startup_df['Date'] = pd.to_datetime(startup_df[date_col], errors='coerce', dayfirst=True)
startup_df = startup_df[startup_df['Date'].notnull()]
startup_df['Year'] = startup_df['Date'].dt.year
startup_df.dropna(subset=['AmountUSD'], inplace=True)
startup_df = startup_df[startup_df['AmountUSD'] > 0]

**ICT DATA CLEANING**

In [ ]:
def safe_to_numeric(s):
    return pd.to_numeric(s.replace('%', '', regex=True).replace(',', '', regex=True), errors='coerce')

for c in ict_df.columns:
    if '%' in c or 'count' in c.lower():
        ict_df[c] = safe_to_numeric(ict_df[c])

ict_df['Year'] = pd.to_numeric(ict_df['Year'], errors='coerce').astype('Int64')
ict_df['City'] = ict_df['City'].astype(str).str.strip()

**FEATURE ENGINEERING**

In [ ]:
ict_df['ICT_CompositeScore'] = ict_df[
    ['Household Internet Access (%)',
     'Fixed Broadband Subscriptions (%)',
     'Wireless Broadband Subscriptions (%)',
     'Wireless Broadband Coverage 3G (%)',
     'Wireless Broadband Coverage 4G (%)',
     'Open data (%)',
     'e-Government (%)']
].mean(axis=1)

**GLOBAL DATA ANALYSIS**

In [ ]:
global_top_countries = global_df['Country'].value_counts().reset_index()
global_top_countries.columns = ['Country', 'CompanyCount']

fig = px.bar(
    global_top_countries.head(15),
    x='Country',
    y='CompanyCount',
    title='Top 15 Countries by Number of Tech Companies',
    text='CompanyCount'
)
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Country', yaxis_title='Number of Companies')
fig.show()

global_by_sector = (
    global_df.groupby('Sector')['MarketCap_BillionUSD']
    .agg(['count', 'sum', 'median'])
    .reset_index()
    .sort_values('sum', ascending=False)
)

fig = px.bar(
    global_by_sector.head(15),
    x='Sector',
    y='sum',
    title='Top 15 Tech Sectors by Total Market Cap (Billion USD)',
    text='sum'
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

**IN STARTUP DATA ANALYSIS**

In [ ]:
fund_by_year = startup_df.groupby('Year')['AmountUSD'].sum().reset_index()
fig = px.line(
    fund_by_year,
    x='Year',
    y='AmountUSD',
    markers=True,
    title='📈 Indian Startup Funding Growth Over the Years'
)
fig.update_layout(yaxis_title='Total Funding (USD)')
fig.show()

fund_by_city = startup_df.groupby(city_col)['AmountUSD'].sum().reset_index().sort_values('AmountUSD', ascending=False).head(20)
fig = px.bar(
    fund_by_city,
    x=city_col,
    y='AmountUSD',
    title='Top 20 Indian Cities by Total Startup Funding',
    text='AmountUSD'
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

**ICT DATA ANALYSISt**

In [ ]:
top_ict_cities = (
    ict_df.groupby('City')['ICT_CompositeScore']
    .mean()
    .reset_index()
    .sort_values('ICT_CompositeScore', ascending=False)
    .head(15)
)
fig = px.bar(
    top_ict_cities,
    x='City',
    y='ICT_CompositeScore',
    title='Top 15 Indian Cities by ICT Composite Score',
    text='ICT_CompositeScore'
)
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

**CORRELATION BETWEEN ALL THREE OF THEM**

In [ ]:
city_year_funding = startup_df.groupby([city_col, 'Year'])['AmountUSD'].sum().reset_index()
ict_for_merge = ict_df[['City', 'Year', 'ICT_CompositeScore']].drop_duplicates()

merged = pd.merge(city_year_funding, ict_for_merge, left_on=[city_col, 'Year'], right_on=['City', 'Year'], how='left')
merged['ICT_CompositeScore'] = merged['ICT_CompositeScore'].fillna(merged['ICT_CompositeScore'].mean())

fig = px.scatter(
    merged,
    x='ICT_CompositeScore',
    y='AmountUSD',
    color='City',
    size='AmountUSD',
    title='Startup Funding vs ICT Composite Score (City-Year)'
)
fig.show()

corr_value = merged[['ICT_CompositeScore', 'AmountUSD']].corr().iloc[0, 1]
print(f"\nCorrelation between ICT development and startup funding: {corr_value:.3f}")


Correlation between ICT development and startup funding: 0.085


**FORECAST (PREDICTION)**

In [ ]:
X = fund_by_year[['Year']]
y = fund_by_year['AmountUSD']
model = LinearRegression()
model.fit(X, y)
future_years = pd.DataFrame({'Year': range(fund_by_year['Year'].max() + 1, fund_by_year['Year'].max() + 6)})
future_years['PredictedFundingUSD'] = model.predict(future_years[['Year']])

fig = go.Figure()
fig.add_trace(go.Scatter(x=fund_by_year['Year'], y=fund_by_year['AmountUSD'], mode='lines+markers', name='Actual Funding'))
fig.add_trace(go.Scatter(x=future_years['Year'], y=future_years['PredictedFundingUSD'], mode='lines+markers', name='Predicted Funding'))
fig.update_layout(title='📉 Indian Startup Funding Forecast (Next 5 Years)', xaxis_title='Year', yaxis_title='Funding (USD)')
fig.show()

print("\n📈 Predicted Funding for Next 5 Years:")
display(future_years)


📈 Predicted Funding for Next 5 Years:


,Year,PredictedFundingUSD
0,2021,3.463978e+09
1,2022,2.641901e+09
2,2023,1.819823e+09
3,2024,9.977447e+08
4,2025,1.756667e+08


**PREDICTION ANALYSIS**

In [ ]:
y_pred = model.predict(X)

pred_col = [col for col in future_years.columns if 'pred' in col.lower()][0]

historical_years = X['Year']
historical_actual = y
historical_predicted = y_pred
future_predicted = future_years[pred_col]
future_year_vals = future_years['Year']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=historical_years,
    y=historical_actual,
    mode='lines+markers',
    name='Actual Funding',
    line=dict(width=3)
))

fig.add_trace(go.Scatter(
    x=historical_years,
    y=historical_predicted,
    mode='lines+markers',
    name='Predicted (Model)',
    line=dict(dash='dot', width=2)
))

fig.add_trace(go.Scatter(
    x=future_year_vals,
    y=future_predicted,
    mode='lines+markers',
    name='Forecasted (Next 5 Years)',
    line=dict(color='green', width=3, dash='dash')
))

fig.update_layout(
    title='Indian Tech Industry Funding Trend: Actual vs Predicted vs Forecast',
    xaxis_title='Year',
    yaxis_title='Funding Amount (USD)',
    template='plotly_white',
    legend=dict(title='Legend', orientation='h', y=-0.2),
    title_font=dict(size=20, color='black'),
    width=950,
    height=550
)

fig.show()


**GLOBAL v/s INDIA ANALYSIS**

In [ ]:
global_total_marketcap = global_df['MarketCap_BillionUSD'].sum()
india_total_funding = fund_by_year['AmountUSD'].sum()

india_total_funding_billion = india_total_funding / 1e9

fig = go.Figure(data=[
    go.Bar(
        x=['Global Tech Industry', 'Indian Tech Startups'],
        y=[global_total_marketcap, india_total_funding_billion],
        text=[f"{global_total_marketcap:.1f} Bn USD", f"{india_total_funding_billion:.2f} Bn USD"],
        textposition='auto',
        marker=dict(color=['#1f77b4', '#2ca02c'])
    )
])

fig.update_layout(
    title='🌍 Global vs 🇮🇳 Indian Tech Industry Comparison',
    xaxis_title='Region',
    yaxis_title='Total Valuation / Funding (Billion USD)',
    template='plotly_white',
    title_font=dict(size=22, color='black'),
    width=850,
    height=500
)

fig.show()